# Keypoint detection, Feature description and matching

In this notebook, I demonstrate how to use OpenCV's feature detection, description and matching framework.

## Compiler parameters

Set my Jypyter environment for the use of [OpenCV](https://www.opencv.org/) in a [C++ notebook](https://github.com/jupyter-xeus/xeus-cling). You don't need this line when you write yur own C++ programs. I need it to set my interactive compiler ([Cling](https://root.cern/cling/)). For your own program, use [CMake](https://www.cmake.org/).

In [26]:
#include "../../../includeLibraries.h"

## Header inclusion for C++

In [27]:
#include <iostream>
#include <vector>
#include <stdexcept>
// #include <sstream>
// #include <string>
#include <limits>
#include <opencv2/opencv.hpp>

## Add the namespaces

In [28]:
using namespace std;

In [29]:
using namespace cv;

## Open two images

### Left image

![Left image](../_IGP8266.JPG)
    
### Right image

![Right image](../_IGP8267.JPG)    

In [30]:
Mat img1 = imread("../_IGP8266.JPG", IMREAD_COLOR);
Mat img2 = imread("../_IGP8267.JPG", IMREAD_COLOR);

### Any error? Is the data loaded?

In [31]:
if (img1.empty())
{
    throw runtime_error("img1 is not loaded, the program will terminate");
}

if (img2.empty())
{
    throw runtime_error("img2 is not loaded, the program will terminate");
}


## 1. Detect

Create a feature detector, here a ORB feature detector.

In [32]:
Ptr<FeatureDetector> detector = ORB::create();

Detect keypoints in `img1` and `img2`.

In [33]:
vector<KeyPoint> keypoints1, keypoints2;
detector->detect(img1, keypoints1);
detector->detect(img2, keypoints2);

## 2. Describe

Create a feature descriptor, here a ORB feature descriptor

In [34]:
Ptr<DescriptorExtractor> extractor = ORB::create();

Create the feature vector for the keypoints.

In [35]:
Mat descriptors1, descriptors2;
extractor->compute(img1, keypoints1, descriptors1);
extractor->compute(img2, keypoints2, descriptors2);

## 3. Match

Match keypoints in `img1` and `img2` by comparing their corresponding feature vectors. Here we use a brute-force algorithm and the L2-norm.

In [36]:
BFMatcher matcher(NORM_L2);
vector<DMatch> matches;
matcher.match(descriptors1, descriptors2, matches);

## 4. Find the good matches

Filter the matches (only keep those with a small distance). 1st find the min and max distances between keypoints

In [37]:
double max_dist = -numeric_limits<double>::max();
double min_dist = numeric_limits<double>::max();

for (int i = 0; i < matches.size(); i++ )
{
    double dist = matches[i].distance;
    if( dist < min_dist ) min_dist = dist;
    if( dist > max_dist ) max_dist = dist;
}

Use only "good" matches (i.e. whose distance is less than a threshold, e.g. the middle distance).

In [38]:
vector<DMatch> good_matches;
double mid_distance = min_dist + (max_dist - min_dist) / 2.0;

for (int i = 0; i < matches.size(); i++ )
{
    if (matches[i].distance < mid_distance)
    {
        good_matches.push_back(matches[i]);
    }
}

## 5. Draw the results

In [39]:
Mat img_matches;
drawMatches(img1, keypoints1, img2, keypoints2, good_matches, img_matches);
imshow("matches", img_matches);
waitKey(0);

We could control the threshold using a trackbar.

![Matches](../match1.png)

Create a callback for the trackbar to control the threshold

In [40]:
int slider_value = 50;

In [41]:
void callback(int, void*)
{
    vector<DMatch> good_matches;
    double threshold_distance = min_dist + (max_dist - min_dist) * slider_value / 100.0;

    for (int i = 0; i < matches.size(); i++ )
    {
        if (matches[i].distance < threshold_distance)
        {
            good_matches.push_back(matches[i]);
        }
    }
    Mat img_matches;
    drawMatches(img1, keypoints1, img2, keypoints2, good_matches, img_matches);
    imshow("matches", img_matches);
}

In [42]:
createTrackbar("Threshold: ", "matches", &slider_value, 100, callback);
imshow("matches", img_matches);
waitKey(0);
destroyAllWindows();

When we increase the threshold there are wrong matches (called false positives).

![Matches with false positives](../match2.png)
    
When we decrease the threshold the false positives disapear as only strong matches remain.

![Strong matches](../match3.png)